In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from src.modelling.model import gen_vocab, generate_text_sequences
from src.datapipeline.datapipeline import Datapipeline

%load_ext autoreload
%autoreload 2

In [2]:
# dpl = Datapipeline(r'./data/raw/realdonaldtrump.csv')
# dpl.transform()
# train, val = dpl.split_data()

In [3]:
# model = load_model('model/trump_bot_bidirstack.h5')

In [4]:
# vocab, vectorizer = gen_vocab(train.values)

In [5]:
# x_val, y_val = generate_text_sequences(val.values, 5, vocab)

In [6]:
# x_input_val = vectorizer.apply(list(map(lambda x: ' '.join(x), x_val)))
# x_input_val = tf.gather(x_input_val, [0,1,2,3,4], axis=1)

Instructions for updating:
Please use `layer.__call__` method instead.


In [7]:
# y_pred_val = model.predict(x_input_val)

In [8]:
# index_label = np.argmax(y_pred_val, axis = 1)

# output_word = []

# for idx in index_label:
#     output_word.append(vocab[idx])

In [9]:
# y_pred_val.shape

(74079, 14400)

In [10]:
# [' '.join(x_val[0])]

['defunding police would be good']

In [2]:
def initialize(data_path, model_path):
    dpl = Datapipeline(data_path)
    dpl.transform()
    train, val = dpl.split_data()
    model = load_model(model_path)
    vocab, vectorizer = gen_vocab(train.values)
    
    return train, val, model, vocab, vectorizer

In [3]:
def tokenize(val, vocab, vectorizer):
    
    x_val, y_val = generate_text_sequences(val.values, 5, vocab)
    x_input_val = vectorizer.apply(list(map(lambda x: ' '.join(x), x_val)))
    x_input_val = tf.gather(x_input_val, [0,1,2,3,4], axis=1)
    
    return x_input_val

In [4]:
def generate_next_word(phrase, vectorizer, model, vocab):
    # if phrase is list of tokens
    if type(phrase) == list:
        x_input_val = vectorizer.apply([' '.join(phrase)])
    # if phrase is string
    elif type(phrase) == str:
        x_input_val = vectorizer.apply([phrase])
    x_input_val = tf.gather(x_input_val, [0,1,2,3,4], axis=1)
    prob_ = model.predict(x_input_val)
    idx = np.argmax(prob_)
    return vocab[idx]

# generate_single_tweet('the trade deficit rose to')

In [5]:
def generate_tweet(phrase, max_char=140):
    word = phrase[-1]
    char_count = 0
    tweet_range = np.random.randint(15,25)
    while word != '' and char_count <= max_char and len(phrase) < tweet_range:
        input_phrase = phrase[-5:]
        word = generate_next_word(input_phrase, vectorizer, model, vocab)
        if word == '':
            break
        elif word =='a' and input_phrase[-1] == 's' and input_phrase[-2] == 'u':
            word = 'usa'
            phrase.pop()
            phrase.pop()

        phrase.append(word)
        char_count = len(' '.join(phrase))
        
    return ' '.join(phrase)

In [11]:
train, val, model, vocab, vectorizer = initialize('./data/raw/realdonaldtrump.csv', './model/trump_bot.h5')
# x_input_val = tokenize(val, vocab, vectorizer)
phrase = "the white house our country".split()[:5] # ['the', 'white', 'house', 'our', 'country']
generate_tweet(phrase)

'the white house our country is doing a great job for the people of the united states and the'

In [13]:
vectorizer